# **<span> <font color='#b8e1f1'>VALE - SOFI - VIC</font> </span>** <font color='#p8e2l1'>- Versión 2</font>

In [1]:
import pandas as pd
import numpy as np
from copy import deepcopy
import matplotlib.pyplot as plt
from scipy.signal import hilbert, stft

# Clase Info 

In [2]:
class Info():
    """ Clase para almacenar información acerca del registro de datos.
       Esta clase se comporta como un diccionario. """

    def __init__(self, ch_names:list= None, ch_types:list= "unknown", bads:list= None, sfreq:float= 512, description:str= None, experimenter:str= "No data", subject_info:dict= None):
        """Genera un objeto Info().

        Parameters:
            ch_names (list of str, optional): Lista con los nombres de los canales.   
            ch_types (str or list of str, optional): Tipo de cada canal (ej: "eeg", "ecg", "emg") o un único tipo para todos.  
            bads (list of str, optional): Lista de canales marcados como “malos”.  
            sfreq (float, optional): Frecuencia de muestreo en Hz (por defecto 512 Hz).  
            description (str, optional): Descripción sobre el registro de datos. 
            experimenter (str, optional): Nombre del experimentador.  
            subject_info (dict, optional): Información adicional del sujeto.

        Raises:
            ValueError: Si "ch_names" y "ch_types" son listas de distinta longitud."""

        if isinstance(ch_types, list) and ch_names is not None:
            if len(ch_types) != len(ch_names):
                raise ValueError("Si 'ch_types' es lista, debe tener la misma longitud que 'ch_names'.")
        
        self._data = {
            'ch_names': ch_names,
            'ch_types': ch_types,
            'bads': bads,
            'sfreq': sfreq,
            'description': description,
            'experimenter': experimenter,
            'subject_info': subject_info
                     }
        
    @property
    def ch_names(self):
        """ """
        return self._data['ch_names']
    
    @property
    def ch_types(self):
        """ """
        return self._data['ch_types']
    
    @property
    def bads(self):
        """ """
        return self._data['bads']
    
    @property
    def sfreq(self):
        """ """
        return self._data['sfreq']
    
    @property
    def description(self):
        """ """
        return self._data['description']
    
    @property
    def experimenter(self):
        """ """
        return self._data['experimenter']
    
    @property
    def subject_info(self):
        """ """
        return self._data['subject_info']

    def __contains__(self, key:str) -> bool: 
        """Permite verificar si una clave está presente en el objeto.
        
        Parameters:
            key (str): Clave a verificar existencia."""
        
        return True if key in self._data else False

    def __getitem__(self, key:str) -> str:
        """Permite acceder a elementos como si fuera un diccionario.
        
        Parameters:
            key (str): Clave a acceder.
            
        Raises:
            KeyError: Si la clave no existe en el objeto."""
        
        try:
            return self._data[key]
        
        except KeyError:
            raise KeyError(f"La clave '{key}' no existe en Info.")

    def __len__(self) -> int:
        """Devuelve la cantidad de elementos almacenados."""
        
        return len(self._data)

    def get(self, key:str):
        """Obtiene el valor de una clave específica.
        
        Parameters:
            key (str): Clave a acceder."""
        
        return self._data.get(key)

    def keys(self):
        """Devuelve un iterador con las claves del objeto."""
        
        return self._data.keys()

    def items(self):
        """Devuelve un iterador con los pares (clave, valor) almacenados."""
        
        return self._data.items()

    def rename_channels(self, mapping:dict):
        """Permite renombrar canales de forma segura.

        Parameters:
            mapping (dict): Diccionario que mapea nombres antiguos de canales a nombres nuevos.

        Raises:
            ValueError: Si alguna clave de mapping no existe en 'ch_names', o si los nuevos nombres introducen duplicados."""

        for key in mapping:
            if key not in self._data['ch_names']:
                raise ValueError(f"No existe un canal con el nombre '{key}' en ch_names.")
    
        new_ch_names = [mapping.get(name, name) for name in self._data['ch_names']]
        
        if len(new_ch_names) != len(set(new_ch_names)):
            raise ValueError("El renombrado generaría nombres duplicados en 'ch_names'.")

        new_bads = [mapping.get(bad, bad) for bad in self._data['bads']]

        self._data['ch_names'] = new_ch_names
        self._data['bads'] = new_bads

### Prueba de la Clase Info

In [3]:
# ***** Imprimimos información sobre la clase *****
# help(Info)                    

# ***** Generando un objeto nuevo *****
canales = [f"C{i+1}" for i in range(3)]
   
info = Info(
    ch_names= canales,
    ch_types= ['eeg'] * len(canales),
    bads= ['C2'],
    sfreq= 512,
    description= "Registro EEG para análisis de patrones ERDS",
    experimenter= "MSc. BALDEZZARI Lucas",
    subject_info= {'edad': 29, 'sexo': 'M'} 
    )

# ***** Acceso a propiedades ***** 
print(info.ch_names)            # → ['C1', 'C2', 'C3']
print(info.ch_types)            # → 512
print(info.bads)                # → ['C2']
print(info.sfreq)               # → 512
print(info.description)         # → Registro EEG para análisis de patrones ERDS
print(info.experimenter)        # → "MSc. BALDEZZARI Lucas"
print(info.subject_info)        # → {'edad': 29, 'sexo': 'M'} 

# ***** Usando el método __contains__() ****** (Verificamos si las keys existen)
print('sfreq' in info)          # → True
print('unknown_key' in info)    # → False

# ***** Usando el método __getitem__() ******
print(info["sfreq"])            # → 512
print(info["experimenter"])     # → MSc. BALDEZZARI Lucas

# ***** Usando el método __len__() ******
print(len(info))                # → 7     

# ***** Usando get() *****
print(info.get("sfreq"))        # → 512

# ***** Usando keys() *****
print(info.keys())              # → dict_keys([···])

# ***** Usando items() *****
print(info.items())             # → dict_items([···])

# ***** Usando rename_channels() ****** (Renombramos canales)
mapping = {'C1': 'Fz', 'C2': 'Cz', 'C3': 'Pz'}
info.rename_channels(mapping)

# Verificar resultados
print(info['ch_names'])       # → ['Fz', 'Cz', 'Pz']
print(info['bads'])           # → ['Cz']

['C1', 'C2', 'C3']
['eeg', 'eeg', 'eeg']
['C2']
512
Registro EEG para análisis de patrones ERDS
MSc. BALDEZZARI Lucas
{'edad': 29, 'sexo': 'M'}
True
False
512
MSc. BALDEZZARI Lucas
7
512
dict_keys(['ch_names', 'ch_types', 'bads', 'sfreq', 'description', 'experimenter', 'subject_info'])
dict_items([('ch_names', ['C1', 'C2', 'C3']), ('ch_types', ['eeg', 'eeg', 'eeg']), ('bads', ['C2']), ('sfreq', 512), ('description', 'Registro EEG para análisis de patrones ERDS'), ('experimenter', 'MSc. BALDEZZARI Lucas'), ('subject_info', {'edad': 29, 'sexo': 'M'})])
['Fz', 'Cz', 'Pz']
['Cz']


# Clase Anotaciones

In [4]:
class Anotaciones:
    """ Clase para almacenar y gestionar anotaciones (eventos) en registros fisiológicos.
        Permite la adición, eliminación, búsqueda, recuperación y persistencia en CSV. """

    def __init__(self, onset:list= None, duration:list= None, description:list= None):
        """Inicializa la clase con los datos de las anotaciones.

        Parameters:
            onset (list of float, optional): Lista de tiempos de inicio de eventos (en segundos).
            duration (list of float, optional): Lista de duraciones de eventos (en segundos).
            description (list of str, optional): Lista de descripciones de eventos.

        Raises:
            ValueError: Si las listas "onset", "duration" y "description" tienen longitudes distintas o falta alguna de ellas cuando se pasa "onset"."""
        
        # Caso: Se proporcionan listas "onset"/"duration"/"description"
        if onset is not None:
            
            if duration is None or description is None:
                raise ValueError("Debe proporcionar 'onset', 'duration' y 'description' juntos.")
                   
            if not (len(duration) == len(description) == len(onset)):
                raise ValueError("Las listas 'onset', 'duration' y 'description' deben tener la misma longitud.")
            
            self.df_eventos = pd.DataFrame({
                                        'onset': onset,
                                        'duration': duration,
                                        'description': description
                                        })
        
        # Caso: Sin datos
        else:
            self.df_eventos = pd.DataFrame(columns= ['onset', 'duration', 'description'])

    def add(self, onset:float, duration:float, description:str):
        """Agrega una nueva anotación.

        Parameters:
            onset (float): Tiempo de inicio del evento (en segundos).
            duration (float): Duración del evento (en segundos).
            description (str): Descripción del evento."""
        
        if not isinstance(onset, float) or not isinstance(duration, float):
            raise TypeError("Los valores 'onset' y 'duration' deben ser numéricos.")
        
        if not isinstance(description, str):
            raise TypeError("La descripción debe ser una cadena de texto.")
        
        nueva = pd.DataFrame([{
            'onset': onset,
            'duration': duration,
            'description': description
                            }]) 
        
        self.df_eventos = pd.concat([self.df_eventos, nueva], ignore_index=True)

    def remove(self, index:int = None, description:str = None):
        """Elimina una anotación por índice o descripción exacta.
        
        Parameters:
            index (int): Índice del evento.
            description (str): Descripción del evento.
        
        Raises:
            ValueError: Si el índice no existe, o la descripción no se encuentra."""
    
        if index is not None:
            
            if index in self.df_eventos.index:
                self.df_eventos.drop(index, inplace=True)
            
            else:
                raise ValueError(f"Índice {index} no existe en las anotaciones")
        
        elif description is not None:
            
            mask = self.df_eventos['description'] != description
            
            if not mask.any():
                raise ValueError(f"No se encontró ninguna anotación con descripción '{description}'")
           
            self.df_eventos = self.df_eventos.loc[mask]
        
        else:
            raise ValueError("Debe especificar index o description para eliminar")

        self.df_eventos.reset_index(drop=True, inplace=True)

    def get_annotations(self) -> pd.DataFrame:
        """Devuelve un DataFrame con todas las anotaciones."""
        
        return self.df_eventos.copy()

    def find(self, description:str) -> pd.DataFrame:
        """Busca y devuelve las anotaciones que coincidan con una descripción específica.
        
        Parameters:
            description (str): Descripción del evento."""
        
        coincidencias  = self.df_eventos[self.df_eventos['description'] == description]
        
        return coincidencias

    def save(self, filepath: str):
        """Guarda las anotaciones en un archivo CSV.
        
        Parameters:
            filepath (str): Ruta del archivo CSV con las anotaciones."""
        
        self.df_eventos.to_csv(filepath, index=False)

    def load(self, filepath: str):
        """Carga anotaciones desde un archivo CSV.
           
        Parameters:
            filepath (str): Ruta del archivo CSV con las anotaciones."""
        
        df_cargado = pd.read_csv(filepath)
        
        for columna in ["onset", "duration", "description"]:
            
            if columna not in df_cargado.columns:
                raise ValueError(f"El CSV no contiene la columna requerida '{columna}'")
        
        self.df_eventos = df_cargado.reset_index(drop=True)

### Prueba de la Clase Anotaciones

In [5]:
# ***** Instanciamos un objeto Anotaciones *****

anotaciones = Anotaciones(
                           onset= [5.0, 12.5, 20.0],
                           duration= [2.0, 3.0, 3.5],
                           description= ["Inicio_Experimento", "Evento_1", "Evento_2"]
                         )

In [6]:
# ***** Imprimimos información sobre la clase *****
# help(Anotaciones) 

# ***** Usando get_annotations() *****
print(anotaciones.get_annotations())

#      onset   duration       description
#  0     5.0     2.00       Inicio_Experimento
#  1     12.5    3.00       Evento_1
#  2     20.0    3.50       Evento_2

# ***** Usando add() ***** (Agregamos una nueva anotación / evento)
anotaciones.add(onset= 30.0, duration= 2.85, description= "Evento_3")
print("\n",anotaciones.get_annotations())

#      onset   duration       description
#  0     5.0     2.00       Inicio_Experimento
#  1     12.5    3.00       Evento_1
#  2     20.0    3.50       Evento_2
#  3     30.0    2.85       Evento_3

# ***** Usando remove() ***** (Eliminamos un evento por descipción / índice)
anotaciones.remove(description= "Evento_1")
print("\n",anotaciones.get_annotations())

# 0 5.0 2.00 Inicio_Experimento
# 1 20.0 3.50 Evento_2
# 2 30.0 2.85 Evento_3

# ***** Usando find() ***** (Buscando anotaciones)
anotaciones.add(onset=40.0, duration=3.1, description="Evento_3")
print("\n",anotaciones.find("Evento_3"))

# onset duration description
# 2 30.0 2.85 Evento_3
# 3 40.0 3.10 Evento_3

# ***** Usando save() *****
# anotaciones.save("mis_anotaciones.csv")

# ***** Usando load() *****
# anotaciones.load("mis_anotaciones.csv")

   onset  duration         description
0    5.0       2.0  Inicio_Experimento
1   12.5       3.0            Evento_1
2   20.0       3.5            Evento_2

    onset  duration         description
0    5.0      2.00  Inicio_Experimento
1   12.5      3.00            Evento_1
2   20.0      3.50            Evento_2
3   30.0      2.85            Evento_3

    onset  duration         description
0    5.0      2.00  Inicio_Experimento
1   20.0      3.50            Evento_2
2   30.0      2.85            Evento_3

    onset  duration description
2   30.0      2.85    Evento_3
3   40.0      3.10    Evento_3


# Clase RawSignal

In [7]:
class RawSignal:
    """ Clase para manejar señales fisiológicas en formato NumPy.
        Este constructor permite inicializar el objeto "RawSignal" a partir de un array de datos, con información adicional de los canales y el índice de la primera muestra. """

    def __init__(self, data:np.ndarray, sfreq:float, info:object = None, first_samp:int= 0, anotaciones:object= None):
        """Inicializa una instancia de la clase RawSignal.
        
        Parameters:
            data (np.ndarray): Matriz de datos con forma "(n_canales , n_muestras)".
            sfreq (float): Frecuencia de muestreo en Hz
            info (Info object, optional): Información adicional sobre la señal, el diccionario contiene información relevante como el tipo de señal, la frecuencia de muestreo y los eventos presentes en la señal.
            first_samp (int, optional): Índice del primer muestreo a utilizar (default es 0).
            anotaciones (Anotaciones object, optional): Objeto de tipo Anotaciones que almacena eventos asociados a la señal y al experimento.
        
        Raises:
            ValueError: Si el array "data" no tiene la forma "(n_canales , n_muestras)".
            ValueError: Si el índice "first_samp" está fuera del rango de la señal."""
        
        #### Agregar Verificaciones ####
        
        self.data = data.copy()
        self.sfreq = sfreq
        self.info = info
        self.first_samp = first_samp
        self.anotaciones = anotaciones

    def get_data(self, picks:list= None, start:float= 0.0, stop:float= None, reject:float= None, times:bool= False):
        """Obtiene muestras de la señal en un rango temporal dado.

        Parameters:
            picks (str or array_like, optional): Canales (nombres o índices) a extraer. Si es "None", se seleccionan todos los canales.
            start (float, optional): Tiempo inicial en segundos para extraer muestras.
            stop (float, optional): Tiempo final en segundos para extraer muestras (default None, que significa hasta el final de la señal).
            reject (float, optional): Valor pico a pico de umbral para rechazar canales. Si una muestra supera este umbral, el canal se descarta.
            times (bool, optional): Si "True" se retorna también el vector de tiempos asociado a las muestras.

        Returns:
            data_sel (np.ndarray): Matriz con los datos seleccionados (n_canales_sel x n_muestras_sel).
            times_vec (np.ndarray): Vector de tiempos (solo si "times= True").
        
        Raises:
            ValueError: Si los índices seleccionados están fuera de rango."""
                    
        n_ch, n_samps = self.data.shape
        
        # Selección de canales
        if picks is None:
            idx_ch = np.arange(n_ch)
        
        else:
            # si nombres
            if all(isinstance(p, str) for p in picks):
                names = self.info['ch_names']
                idx_ch = [names.index(p) for p in picks]
            else:
                idx_ch = np.array(picks, dtype=int)
            # validar rango
            if any((i < 0 or i >= n_ch) for i in idx_ch):
                raise ValueError("Algunos índices de canal fuera de rango.")

        # Conversión de tiempos a muestras
        s0 = self.first_samp + int(start * self.sfreq)
        s1 = n_samps if stop is None else self.first_samp + int(stop * self.sfreq)
        if not (self.first_samp <= s0 < s1 <= n_samps):
            raise ValueError("Valores de start/stop fuera de rango de la señal.")

        # Extracción
        data_sel = self.data[idx_ch, s0:s1]

        # Rechazo por amplitud
        if reject is not None:
            ptp = data_sel.ptp(axis=1)
            mask = ptp <= reject
            data_sel = data_sel[mask, :]

        # Vector de tiempos
        if times:
            times_vec = (np.arange(s0, s1) - self.first_samp) / self.sfreq
            return data_sel, times_vec
        return data_sel

    def drop_channels(self, ch_names):
        """
        Elimina canales por nombre, devuelve nueva instancia RawSignal.
        """
        # Índices a eliminar
        all_names = self.info['ch_names']
        to_drop = [all_names.index(ch) for ch in ch_names]
        mask = [i for i in range(self.data.shape[0]) if i not in to_drop]

        # Nuevo data y nueva info
        new_data = self.data[mask, :]
        new_info = deepcopy(self.info)
        new_info._data['ch_names'] = [all_names[i] for i in mask]
        new_info._data['ch_types'] = [new_info._data['ch_types'][i] for i in mask]

        # Anotaciones se conservan
        return RawSignal(new_data, self.sfreq, info=new_info,
                         anotaciones=deepcopy(self.anotaciones),
                         first_samp=self.first_samp)

    def crop(self, tmin=0.0, tmax=None):
        """Elimina uno o más canales a partir de ch_names
        
        Parameters:        
            ch_names (array_like): Nombres de canales a eliminar
        
        Returns:
            RawSignal object"""
            
        n_ch, n_samps = self.data.shape
        start_idx = self.first_samp + int(tmin * self.sfreq)
        end_idx = n_samps if tmax is None else self.first_samp + int(tmax * self.sfreq)
        if not (self.first_samp <= start_idx < end_idx <= n_samps):
            raise ValueError("tmin/tmax fuera de rango.")

        # Extraer segmento
        new_data = self.data[:, start_idx:end_idx]

        # Ajustar annotations
        df = self.anotaciones.get_annotations()
        # filtramos eventos dentro [tmin, tmax)
        df2 = df[(df['onset'] >= tmin) & (df['onset'] < (tmax or df['onset'].max()))].copy()
        # reescalamos onset restando tmin
        df2['onset'] = df2['onset'] - tmin
        new_anots = Anotaciones(annotations=df2)

        # Info se mantiene (first_samp adaptada)
        return RawSignal(new_data, self.sfreq, info=deepcopy(self.info),
                         anotaciones=new_anots, first_samp=0)
        
    def describe(self):
        names = self.info['ch_names']
        types = self.info['ch_types']
        stats = []
        for i in range(self.data.shape[0]):
            channel = self.data[i, :]
            vals = {'name': names[i],
                    'type': types[i],
                    'min': float(channel.min()),
                    'Q1': float(np.percentile(channel, 25)),
                    'mediana': float(np.percentile(channel, 50)),
                    'Q3': float(np.percentile(channel, 75)),
                    'max': float(channel.max())}
            stats.append(vals)
        import pandas as pd
        return pd.DataFrame(stats)
    
    def filter(self, l_freq, h_freq, notch_freq=50.0, order=4, fir_window='hamming'):
        # Validar parámetros
        if l_freq <= 0 or h_freq <= 0 or l_freq >= h_freq or h_freq >= self.sfreq / 2:
            raise ValueError("Frecuencias l_freq y h_freq no válidas.")
        if notch_freq <= 0:
            raise ValueError("notch_freq debe ser positivo.")

        # Diseño filtro pasabanda IIR Butterworth
        nyq = 0.5 * self.sfreq
        low = l_freq / nyq
        high = h_freq / nyq
        b, a = butter(order, [low, high], btype='band')
        filtered = filtfilt(b, a, self.data, axis=1)

        # Diseño notch
        q = 30.0
        b_n, a_n = iirnotch(notch_freq / nyq, q)
        filtered = filtfilt(b_n, a_n, filtered, axis=1)

        return RawSignal(filtered, self.sfreq, info=deepcopy(self.info),
                         anotaciones=deepcopy(self.anotaciones),
                         first_samp=self.first_samp)


    def pick(self, picks):
        n_ch = self.data.shape[0]
        all_names = self.info['ch_names']
        if isinstance(picks, str):
            if picks not in all_names:
                raise ValueError(f"Canal '{picks}' no existe.")
            sel = [all_names.index(picks)]
        elif isinstance(picks, slice):
            sel = list(range(n_ch))[picks]
        else:
            sel = []
            for p in picks:
                if isinstance(p, str):
                    if p not in all_names:
                        raise ValueError(f"Canal '{p}' no existe.")
                    sel.append(all_names.index(p))
                else:
                    i = int(p)
                    if i < 0 or i >= n_ch:
                        raise ValueError(f"Índice de canal {i} fuera de rango.")
                    sel.append(i)

        new_data = self.data[sel, :]
        new_info = deepcopy(self.info)
        chn = []
        tp = []
        for i in sel:
            chn.append(new_info._data['ch_names'][i])
            tp.append(new_info._data['ch_types'][i])
        new_info._data['ch_names'] = chn
        new_info._data['ch_types'] = tp

        return RawSignal(new_data, self.sfreq, info=new_info,
                         anotaciones=deepcopy(self.anotaciones),
                         first_samp=self.first_samp)
    
    def set_anotaciones(self, anotaciones):
        """
        Asocia un objeto de tipo 'Anotaciones' a la señal fisiológica.

        Parameters
        ----------
        anotaciones : Anotaciones
            Objeto de la clase 'Anotaciones'.

        Raises
        ------
        TypeError
            Si 'anotaciones' no es instancia de Anotaciones.
        ValueError
            Si alguna anotación está fuera del rango temporal de la señal.
        """
        if not isinstance(anotaciones, Anotaciones):
            raise TypeError("El parámetro debe ser una instancia de Anotaciones.")
        df = anotaciones.get_annotations()
        # Verificar que onset esté dentro de duración de la señal
        max_time = (self.data.shape[1] - self.first_samp) / self.sfreq
        for _, row in df.iterrows():
            if row['onset'] < 0 or row['onset'] > max_time:
                raise ValueError(
                    f"La anotación en {row['onset']}s está fuera del rango [0, {max_time}]"
                )
        self.anotaciones = anotaciones
        
    def plot(self, picks=None, start=0.0, duration=10.0, show_anotaciones=True):
        # Extraer segmento
        stop = start + duration
        if show_anotaciones:
            data_sel, times_vec = self.get_data(picks=picks,
                                               start=start,
                                               stop=stop,
                                               times=True)
        else:
            data_sel = self.get_data(picks=picks,
                                     start=start,
                                     stop=stop,
                                     times=False)
            times_vec = (np.arange(
                int(start*self.sfreq),
                int(stop*self.sfreq)
            ) / self.sfreq)

        # Trazar
        plt.figure()
        for idx in range(data_sel.shape[0]):
            plt.plot(times_vec, data_sel[idx, :], label=self.info['ch_names'][idx])
        plt.xlabel('Time (s)')
        plt.ylabel('Amplitude')
        plt.title('RawSignal')
        plt.legend()
#falta implementar __getitem__()

### Prueba Clase RawDignal

In [9]:
# 1. Datos de prueba: 4 canales, 100 muestras
data = np.arange(4 * 100).reshape(4, 100).astype(float)
sfreq = 50.0  # Hz

# 2. Instanciar RawSignal
rs = RawSignal(data, sfreq)

# 3. Probar get_data
d_all = rs.get_data()  # Todos los datos
d_segment, times = rs.get_data(start=0.5, stop=1.5, times=True)  # 0.5s a 1.5s

# 4. Probar describe
df_stats = rs.describe()

# 5. Probar drop_channels
rs_drop = rs.drop_channels(['Chan1', 'Chan3'])
drop_names = rs_drop.info['ch_names']

# 6. Probar crop
rs_crop = rs.crop(tmin=1.0, tmax=2.0)
crop_shape = rs_crop.data.shape

# 7. Probar pick
rs_pick = rs.pick(['Chan0', 'Chan2'])
pick_names = rs_pick.info['ch_names']

# 8. Mostrar resultados

print("=== Datos del segmento (0.5s–1.5s) ===")
df_segment = pd.DataFrame(
    d_segment,
    index=rs.info['ch_names'],
    columns=np.round(times, 2)
)
print(df_segment)

print("\n=== Estadísticas por canal ===")
print(df_stats)

print(f"\nCanales después de drop_channels: {drop_names}")
print(f"Forma de datos tras crop: {crop_shape}")
print(f"Canales tras pick: {pick_names}")

#9. Probar plot 
#rs.plot(picks=['Chan0','Chan2'], start=0.0, duration=2.0, show_anotaciones=False)
#plt.show()


TypeError: 'NoneType' object is not subscriptable